# Zooming SlowMo
This Colab is based on [this](https://github.com/Mukosame/Zooming-Slow-Mo-CVPR-2020). This Colab was created by [Styler00Dollar](https://github.com/styler00dollar) aka "sudo rm -rf / --no-preserve-root#8353" on discord

Simple tutorial:

- Run cells with these play-buttons that are visible on the left side of the code/text. ```[ ]``` indicate a play-button.

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install everything
!git clone https://github.com/styler00dollar/Colab-Zooming-Slow-Mo
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install pytorch cudatoolkit==10.0.130 torchvision -c pytorch -y
%cd /content/Colab-Zooming-Slow-Mo/codes/models/modules/DCNv2
!python setup.py build develop
!pip install gdown
%cd /content/
!gdown --id 1xeOoZclGeSI1urY6mVCcApfCqOPgxMBK
!pip install opencv-python
!pip install pyyaml

In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Google Drive connected.')

Put your file into Google Drive. The default input path is ```Google Drive/input.mp4```.

This needs a lot of resources, since it does interpolation and 4x upscaling at the same time. Don't input high resolutions. 432x240 seems to work.

Create video without sound:

In [ ]:
# Copy video to Colab and doing interpolation
!cp "/content/drive/My Drive/input.mp4" /content/input.mp4

# Detecting FPS of input file
import os
import cv2
cap = cv2.VideoCapture('/content/input.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
print("Detected FPS: ")
print(fps)

!python /content/Colab-Zooming-Slow-Mo/codes/video_to_zsm.py --video /content/input.mp4 --model /content/Colab-Zooming-Slow-Mo/experiments/pretrained_models/xiang2020zooming.pth

amount_files_extract_dir = len(os.listdir('/content/Colab-Zooming-Slow-Mo/extract'))
amount_files_tmp_dir = len(os.listdir('/content/Colab-Zooming-Slow-Mo/tmp'))

new_fps = fps*(amount_files_tmp_dir/amount_files_extract_dir)

%shell ffmpeg -y -r {new_fps} -f image2 -pattern_type glob -i '/content/Colab-Zooming-Slow-Mo/tmp/*.png' -crf 18 /content/output.mp4
# Copy video back to Google Drive
!cp /content/output.mp4 "/content/drive/My Drive/output.mp4"

[Experimental] Create video with sound

In [ ]:
# Copy video to Colab and doing interpolation
!cp "/content/drive/My Drive/input.mp4" /content/input.mp4

# Detecting FPS of input file
import os
import cv2
cap = cv2.VideoCapture('/content/input.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
print("Detected FPS: ")
print(fps)

%cd /content/Colab-Zooming-Slow-Mo/codes
!python video_to_zsm.py --video /content/input.mp4 --model /content/Colab-Zooming-Slow-Mo/experiments/pretrained_models/xiang2020zooming.pth

amount_files_extract_dir = len(os.listdir('/content/Colab-Zooming-Slow-Mo/extract'))
amount_files_tmp_dir = len(os.listdir('/content/Colab-Zooming-Slow-Mo/tmp'))

new_fps = fps*(amount_files_tmp_dir/amount_files_extract_dir)

%shell ffmpeg -i /content/input.mp4 -acodec copy /content/output-audio.aac
%shell ffmpeg -y -r {new_fps} -f image2 -pattern_type glob -i '/content/Colab-Zooming-Slow-Mo/tmp/*.png' -i /content/output-audio.aac -shortest -crf 18 /content/output.mp4
# Copy video back to Google Drive
!cp /content/output.mp4 "/content/drive/My Drive/output.mp4"

# Preview the result within Colab

Don't try this with big files. It will crash Colab. Small files like 10mb are ok. ~20+mb will crash colab.

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('/content/output.mp4', width=960, height=720)